In [ ]:
!pip install diffusers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 68.8 MB/s eta 0:00:00


In [ ]:
# @title Pipeline Server


In [ ]:
!pip install pyngrok
!pip install flask_cors

In [ ]:
from pyngrok import ngrok
from google.colab import userdata

# Replace 'YOUR_NGROK_AUTHTOKEN' with the authtoken you copied from the ngrok dashboard
authtoken = userdata.get("ngrok_key")
ngrok.set_auth_token(authtoken)

In [ ]:
pip install "Flask[async]"

In [ ]:
import os
import tempfile
from flask import Flask, jsonify, request, send_file
from werkzeug.utils import secure_filename
import torch
# from diffusers import StableVideoDiffusionPipeline
# from diffusers.utils import load_image, export_to_video
from diffusers import I2VGenXLPipeline
from diffusers.utils import export_to_gif, load_image
from pyngrok import ngrok
from flask_cors import CORS
import asyncio

app = Flask(__name__)
CORS(app)  # Enable CORS

# Load the pipeline globally to avoid reloading on every request
# pipeline = StableVideoDiffusionPipeline.from_pretrained(
#     "stabilityai/stable-video-diffusion-img2vid-xt", torch_dtype=torch.float16, variant="fp16"
# )
pipeline = I2VGenXLPipeline.from_pretrained(
    "ali-vilab/i2vgen-xl", torch_dtype=torch.float16, variant="fp16"
)
pipeline.enable_model_cpu_offload()

def generate_video(image_url, input_prompt, input_neg_prompt):
    image = load_image(image_url)
    # image = image.resize((1024, 576))
    image = image.convert("RGB")

    prompt = input_prompt
    negative_prompt = input_neg_prompt

    generator = torch.manual_seed(42)
    # frames = pipeline(image, decode_chunk_size=8, generator=generator, num_frames=15).frames[0]
    # export_to_video(frames, "generated.mp4", fps=7)
    frames = pipeline(
        prompt=prompt,
        image=image,
        num_inference_steps=40,
        negative_prompt=negative_prompt,
        guidance_scale=9.0,
        generator=generator
    ).frames[0]
    export_to_gif(frames, "i2v.gif")
    return "i2v.gif"

@app.route('/generate-video', methods=['POST'])
async def generate_video_endpoint():
    try:
        data = request.json
        image_url = data.get('image_url')
        input_prompt = data.get('input_prompt')
        input_neg_prompt = data.get('input_neg_prompt')

        loop = asyncio.get_event_loop()
        video_path = await loop.run_in_executor(None, generate_video, image_url, input_prompt, input_neg_prompt)

        return jsonify({"status": "success", "video_url": f"/download-video/{video_path}"}), 200

    except Exception as e:
        print(f"Error: {str(e)}")
        return jsonify({"error": "Something went wrong."}), 500

@app.route('/download-video/<filename>', methods=['GET'])
def download_video(filename):
    return send_file(filename, as_attachment=True)


# Terminate any open tunnels if running in a reused environment
ngrok.kill()

# Create a tunnel to the default Flask port 5000
public_url = ngrok.connect(5000)
print(f"Public URL: {public_url}")

if __name__ == '__main__':
    app.run()

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

...

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Public URL: NgrokTunnel: "https://42cd-34-142-131-6.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


  0%|          | 0/40 [00:00<?, ?it/s]

INFO:werkzeug:127.0.0.1 - - [08/Sep/2024 13:15:32] "POST /generate-video HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [08/Sep/2024 13:15:33] "GET /download-video/i2v.gif HTTP/1.1" 200 -


In [ ]:
# @title Test


In [ ]:
import torch
from diffusers import I2VGenXLPipeline
from diffusers.utils import export_to_gif, load_image

pipeline = I2VGenXLPipeline.from_pretrained("ali-vilab/i2vgen-xl", torch_dtype=torch.float16, variant="fp16")
pipeline.enable_model_cpu_offload()

image_url = "https://huggingface.co/datasets/diffusers/docs-images/resolve/main/i2vgen_xl_images/img_0009.png"
image = load_image(image_url).convert("RGB")

prompt = "Papers were floating in the air on a table in the library"
negative_prompt = "Distorted, discontinuous, Ugly, blurry, low resolution, motionless, static, disfigured, disconnected limbs, Ugly faces, incomplete arms"
generator = torch.manual_seed(8888)

frames = pipeline(
    prompt=prompt,
    image=image,
    num_inference_steps=50,
    negative_prompt=negative_prompt,
    guidance_scale=9.0,
    generator=generator
).frames[0]
export_to_gif(frames, "i2v.gif")